<a href="https://colab.research.google.com/github/redoy722/GNN_Attack/blob/main/Orbit_based_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages.
import os
import torch
import networkx as nx
import math
from scipy.sparse import csr_matrix
# Importing required libraries
import numpy as np
from tqdm import tqdm
import scipy.sparse as sp


os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

! pip install deeprobust

1.13.1+cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 21.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 62.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.8.3-cp39-cp39-linux_x86_64.whl size=26528058 sha256=0751aaa43b9accf85726439b10f9cd25545fe80a69896b991f9a04fc8c6d0420
  Stored in directory: /root/.cache/pip/wheels/ca/5d/af/618594ec2f28608c1d6ee7d2b7e95a3e9b06551e3b80a491d6
Successfully built gensim
  Attempting un

**Modified ORCA Algorithm uploaded to the google drive and run from it**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!pip install /content/drive/MyDrive/orca-py2/orca-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./drive/MyDrive/orca-py2/orca-py
  Preparing metadata (setup.py) ... done
  Created wheel for orca: filename=orca-1.0-cp39-cp39-linux_x86_64.whl size=259038 sha256=c9a68e0feb8d1e4959b795850c6abb27808501778822cb9114582c77edae1e84
  Stored in directory: /root/.cache/pip/wheels/e1/c9/7b/b1b626dd74d1f1987fc898c64edbbf74a55200ed0d2ae538cf
Successfully built orca


**Load Data**

In [3]:
########### load clean graph  ###########

from deeprobust.graph.data import Dataset, Dpr2Pyg, Pyg2Dpr

Data = Dataset(root='/tmp/', name='cora') ################### dataset cora ########################

#Data=Dataset(root='/tmp/', name='citeseer') ######################## dataset citeseer ########################

#Data=Dataset(root='/tmp/', name='cora_ml') ###################### dataset cora ml ##############################

Loading cora dataset...
Done!
Selecting 1 largest connected components


In [4]:
######### Deepribust data to pyg data conversion for GraphIn Adj #########
datapy=Data
pyg_data = Dpr2Pyg(datapy) 
print(pyg_data)

Dpr2Pyg()


Processing...
/usr/local/lib/python3.9/dist-packages/deeprobust/graph/data/pyg_dataset.py:48: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  edge_index = torch.LongTensor(dpr_data.adj.nonzero())
Done!


In [5]:
print(pyg_data[0])

Data(x=[2485, 1433], edge_index=[2, 10138], y=[2485], train_mask=[2485], val_mask=[2485], test_mask=[2485])


/usr/local/lib/python3.9/dist-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [6]:
graph_ed=pyg_data[0].edge_index.t()
print(graph_ed)

tensor([[   0, 1084],
        [   0, 1104],
        [   0, 1288],
        ...,
        [2484, 1184],
        [2484, 1255],
        [2484, 1894]])


In [7]:
import numpy
np_graph_ed = graph_ed.numpy()

In [8]:
x=len(pyg_data[0].y)
y=len(np_graph_ed)
xy=[x,y]

In [9]:
with open("graph.in","w") as f:
    f.write("\n".join(" ".join(map(str, x)) for x in (np_graph_ed)))

In [10]:
with open('graph.in', 'r') as file:
    contents = file.read()
    file.close()

with open('graph.in', 'w') as file:
    file.write(' '.join(map(str, xy)))
    file.write("\n")
    file.write(contents)
    file.close()

**graphlet Generation**

In [13]:
!python /content/drive/MyDrive/orca-py2/orca-py/test.py

graph: Graph with 2485 nodes and 5069 edges
Number of nodes: 2485
Number of graphlets: 73
Counts:  [[5, 50, 8, 2, 157, 149, 436, 5, 3, 20, 43, 4, 1, 1, 0, 939, 433, 534, 661, 2121, 1237, 139, 3403, 1, 77, 112, 85, 31, 63, 159, 17, 308, 400, 2, 6, 17, 4, 75, 6, 13, 30, 3, 2, 15, 0, 11, 3, 2, 32, 1, 0, 1, 4, 1, 0, 0, 4, 0, 0, 1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0], [6, 15, 9, 6, 137, 51, 11, 5, 1, 8, 18, 8, 2, 5, 2, 493, 468, 63, 988, 195, 43, 43, 3, 1, 115, 11, 22, 42, 27, 145, 31, 5, 12, 2, 1, 10, 10, 0, 2, 3, 0, 4, 6, 5, 0, 8, 21, 2, 22, 0, 0, 3, 0, 1, 0, 1, 1, 7, 2, 5, 3, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 1, 0], [7, 82, 16, 5, 282, 394, 932, 14, 6, 18, 86, 17, 0, 4, 0, 1662, 1369, 1881, 1045, 5804, 4525, 557, 8369, 4, 105, 183, 266, 106, 75, 266, 234, 418, 782, 15, 10, 30, 15, 271, 18, 8, 0, 37, 10, 28, 3, 13, 0, 0, 76, 7, 0, 5, 1, 10, 0, 0, 2, 0, 0, 0, 0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0], [31, 77, 451, 14, 475, 2184, 77, 4101, 24, 17, 56, 384, 11, 8, 2, 1820, 12791, 2535, 3443, 94

**Read Graphlet Data**

In [14]:
with open('graph.out', 'r') as file:
    data = file.readlines()

In [15]:
int_list = [[int(num) for num in s.strip('[]\n').split(',')] for s in data]
graphlet_features = np.array(int_list)
print(graphlet_features)

[[ 5 50  8 ...  0  0  0]
 [ 6 15  9 ...  0  1  0]
 [ 7 82 16 ...  0  0  0]
 ...
 [ 1  3  0 ...  0  0  0]
 [ 1  3  0 ...  0  0  0]
 [ 1  2  0 ...  0  0  0]]


**Function defination**

In [17]:
def remove_duplicate(lst):
  
  nodelist= lst
  new_list = []
  seen = set()
  
  for item in nodelist:
        if item not in seen:
          seen.add(item)
          new_list.append(item)
  return new_list

In [19]:
########### Orbit based feature modification function ##################

def orbit_based_feature_modification(Gl_features, orgfeat, pub_rate):
  
  totalnode=len(Gl_features)
  nodelstlst=[]
  
  for i in range(totalnode):
    orbits=[23, 33, 44, 69] ######## taking only central orbit
    nodelist=[]

    for num in orbits:
      orbit_num=Gl_features[:, num]
      sorted_indices = np.argsort(orbit_num)[::-1]
      nodelist.append(sorted_indices[i])
    
    node=remove_duplicate(nodelist)
    nodelstlst.append(node)
    
  flat_list = [item for sublist in nodelstlst for item in sublist]
  removed_node_feature_list=remove_duplicate(flat_list)
  
  org_features=orgfeat
  # convert it to a dense matrix
  dense_org_features = org_features.toarray()
  
  budget = math.ceil((totalnode)*pub_rate)
  
  for n in range(budget):
    mod_fet=removed_node_feature_list[n]
    dense_org_features[mod_fet]=np.random.uniform(-1, 1, len(dense_org_features[mod_fet]))
  
  mod_features= sp.csr_matrix(dense_org_features)

  return mod_features

In [24]:
########### Orbit based edge removal function##################

def new_orbit_based_edge_remove(Gl_features, adj, pub_rate):

  if not sp.issparse(adj):
    adj = sp.csr_matrix(adj)
  
  modified_adj = adj.copy().tolil() ##### will use last
  
  orbits=[23, 33, 44, 69]  ######## taking only central orbit

  Gl_features=Gl_features[:, orbits]

  # sum the values of each column
  column_sum = np.sum(Gl_features, axis=0)
  # sort the rows in descending order
  sorted_arr = np.sort(column_sum, axis=0)[::-1]
  # get the indices of the sorted values
  sorted_indices = np.argsort(column_sum)[::-1]
  
  edges = np.array(modified_adj.nonzero()).T
  
  budget= math.ceil((len(edges)/2)*pub_rate)
  
  mylist=[] ####### storing the deletable edge list based on the orbit 
  
  remd_edg_lst=[] ########### removed edge list 
  
  modadjarry=modified_adj.toarray()
  
  # calculate the degree of each node by summing the rows of the matrix
  degrees = np.sum(modadjarry, axis=1)
  
  removed_cnt = 0
  
  no_edr= budget
  
  nooo_edr = budget

  mx_features =Gl_features

  for i in range(len(sorted_indices)):
    
    if nooo_edr == 0:
      break
    
    iorb=sorted_indices[i]
      

    for edge in tqdm(edges):
            n1 = edge[0]
            n2 = edge[1]

            if Gl_features[n1][iorb] >= max(mx_features[n1]) and Gl_features[n2][iorb] >= max(mx_features[n2]):
                mylist.append([n1, n2, Gl_features[n1][iorb]+Gl_features[n2][iorb]])
                

    sssorted_list =sorted(mylist, key=lambda x: x[2], reverse=True) ##### use it
    
    # Using list comprehension to create a new list of lists where the last item is greater than 50
    
    sorted_list = [lst for lst in sssorted_list if lst[-1] >= 50]

    for i in range(0, len(sorted_list), 1):
      
      if removed_cnt < no_edr: 

          als1= sorted_list[i][0:2]
          rls2 = als1[::-1]
          
          if  als1 not in remd_edg_lst and rls2 not in remd_edg_lst:
            if degrees[sorted_list[i][0]] > 1 and degrees[sorted_list[i][1]] > 1:
              modified_adj[sorted_list[i][0], sorted_list[i][1]] = 0.0
              modified_adj[sorted_list[i][1], sorted_list[i][0]] = 0.0
        
              removed_cnt += 1

              remd_edg_lst.append(als1)
              remd_edg_lst.append(rls2)

              degrees[sorted_list[i][0]]= degrees[sorted_list[i][0]]-1
              degrees[sorted_list[i][1]]= degrees[sorted_list[i][1]]-1

    nooo_edr= no_edr - removed_cnt
    print('removed %s edges in the original graph' % removed_cnt)
    print('Rest %s edges need to delete in the original graph' % nooo_edr)
  
    mx_features[:, iorb] = 0

  return modified_adj

**Data Load**

In [21]:
from deeprobust.graph.data import Dataset, PrePtbDataset
# load the prognn splits by using setting='prognn'
# because the attacked graphs are generated under prognn splits

data = Dataset(root='/tmp/', name='cora')

adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
# Load meta attacked data
perturbed_data = PrePtbDataset(root='/tmp/', name='cora', attack_method='meta', ptb_rate=0.05)
perturbed_adj = perturbed_data.adj

Loading cora dataset...
Selecting 1 largest connected components
Dowloading from https://raw.githubusercontent.com/ChandlerBang/Pro-GNN/master/meta/cora_meta_adj_0.05.npz to /tmp/cora_meta_adj_0.05.npz
Loading cora dataset perturbed by 0.05 meta...


/usr/local/lib/python3.9/dist-packages/deeprobust/graph/data/attacked_data.py:135: UserWarning: The pre-attacked graph is perturbed under the data splits provided by ProGNN. So if you are going to verify the attacking performance, you should use the same data splits  (setting='prognn').
  warnings.warn("The pre-attacked graph is perturbed under the data splits provided by ProGNN. So if you are going to verify the attacking performance, you should use the same data splits  (setting='prognn').")


In [25]:
gp_adj=data.adj  ######## assign original grpah adjacency 
orgfeat=data.features ######## assign original grpah feature 
glt_feature= graphlet_features  ######## assign graphlet feature 

edg_pub_rate=0.05
#fet_pub_rate=0.025

mod_adj=new_orbit_based_edge_remove(glt_feature, gp_adj, edg_pub_rate)
#mod_fet= orbit_based_feature_modification(glt_feature, orgfeat, fet_pub_rate)
  

100%|██████████| 10138/10138 [00:00<00:00, 191141.24it/s]


removed 240 edges in the original graph
Rest 14 edges need to delete in the original graph


100%|██████████| 10138/10138 [00:00<00:00, 47137.62it/s]

removed 254 edges in the original graph
Rest 0 edges need to delete in the original graph


**GCN Model**

In [26]:
from deeprobust.graph.defense import GCN

gcn = GCN(nfeat=features.shape[1], nhid=16, nclass=labels.max().item() + 1, dropout=0.5, device='cpu')
gcn = gcn.to('cpu')

gcn.fit(features, adj, labels, idx_train, idx_val) # train on clean graph with earlystopping
gcn.test(idx_test)

gcn.fit(features, perturbed_adj, labels, idx_train, idx_val) # train on poisoned graph
gcn.test(idx_test)

############ ############ GLet Attack without modified feature # ##########

gcn.fit(features, mod_adj, labels, idx_train, idx_val) # train on developed poisoned graph
gcn.test(idx_test)

############ ############ ############ GLet Attack with modified feature ########## ##########

#gcn.fit(mod_fet, mod_adj, labels, idx_train, idx_val) # train on developed poisoned graph
#gcn.test(idx_test)

Test set results: loss= 0.5339 accuracy= 0.8375
Test set results: loss= 0.5404 accuracy= 0.8335
Test set results: loss= 0.5377 accuracy= 0.8295


0.829476861167002

**GCN Jaccard Model**

In [27]:
from deeprobust.graph.defense import GCNJaccard

model = GCNJaccard(nfeat=features.shape[1], nhid=16, nclass=labels.max().item() + 1, dropout=0.5, device='cpu').to('cpu')


model.fit(features, adj, labels, idx_train, idx_val, threshold=0.03)
model.test(idx_test)

############ Meta Attack ##########
model.fit(features, perturbed_adj, labels, idx_train, idx_val, threshold=0.03) # train on poisoned graph
model.test(idx_test)

############ nettacked Attack ##########
#model.fit(features, net_perturbed_adj, labels, idx_train, idx_val, threshold=0.01) # train on poisoned graph
#model.test(idx_test)

############ ############ GLet Attack without modified feature ########## ##########

model.fit(features, mod_adj, labels, idx_train, idx_val, threshold=0.03)
model.test(idx_test)

############ ############ GLet Attack with modified feature ########## ##########

# model.fit(mod_fet, mod_adj, labels, idx_train, idx_val, threshold=0.03)
# model.test(idx_test)

removed 1015 edges in the original graph
=== training gcn model ===
Epoch 0, training loss: 1.9528292417526245
Epoch 10, training loss: 1.076555848121643
Epoch 20, training loss: 0.5327485799789429
Epoch 30, training loss: 0.2858056128025055
Epoch 40, training loss: 0.16181227564811707
Epoch 50, training loss: 0.09233883023262024
Epoch 60, training loss: 0.09063504636287689
Epoch 70, training loss: 0.0917663648724556
Epoch 80, training loss: 0.07766369730234146
Epoch 90, training loss: 0.09218998998403549
Epoch 100, training loss: 0.06998526304960251
Epoch 110, training loss: 0.10645966231822968
Epoch 120, training loss: 0.07869210839271545
Epoch 130, training loss: 0.07522448897361755
Epoch 140, training loss: 0.06595957279205322
Epoch 150, training loss: 0.07643083482980728
Epoch 160, training loss: 0.04123331978917122
Epoch 170, training loss: 0.06549327075481415
Epoch 180, training loss: 0.039611946791410446
Epoch 190, training loss: 0.046531904488801956
=== picking the best model 

0.8048289738430584

**GNN-SVD**

In [28]:
from deeprobust.graph.defense import GCNSVD

# train defense model
model = GCNSVD(nfeat=features.shape[1], nhid=16, nclass=labels.max().item() + 1, dropout=0.5, device='cpu').to('cpu')

model.fit(features, adj, labels, idx_train, idx_val, k=20)
model.test(idx_test)

############ Meta Attack ##########
model.fit(features, perturbed_adj, labels, idx_train, idx_val, k=20)
model.test(idx_test)

############ nettacked Attack ##########
#model.fit(features, net_perturbed_adj, labels, idx_train, idx_val, k=20)
#model.test(idx_test)

############ ############ GLet Attack without modified feature ########## ##########
model.fit(features, mod_adj, labels, idx_train, idx_val, k=20)
model.test(idx_test)

############ ############ GLet Attack with modified feature ########## ##########

# model.fit(mod_fet, mod_adj, labels, idx_train, idx_val, k=20)
# model.test(idx_test)

=== GCN-SVD: rank=20 ===
rank_after = 20
=== training gcn model ===
Epoch 0, training loss: 1.930056095123291
Epoch 10, training loss: 1.1307491064071655
Epoch 20, training loss: 0.6882537007331848
Epoch 30, training loss: 0.4587513208389282
Epoch 40, training loss: 0.38660573959350586
Epoch 50, training loss: 0.3513208031654358
Epoch 60, training loss: 0.33311983942985535
Epoch 70, training loss: 0.30968230962753296
Epoch 80, training loss: 0.3123846650123596
Epoch 90, training loss: 0.26649945974349976
Epoch 100, training loss: 0.26080775260925293
Epoch 110, training loss: 0.23915819823741913
Epoch 120, training loss: 0.24835248291492462
Epoch 130, training loss: 0.22526618838310242
Epoch 140, training loss: 0.21318857371807098
Epoch 150, training loss: 0.2591603100299835
Epoch 160, training loss: 0.20193950831890106
Epoch 170, training loss: 0.19040338695049286
Epoch 180, training loss: 0.18849870562553406
Epoch 190, training loss: 0.20195457339286804
=== picking the best model acco

0.7434607645875252

**RGCN model**

In [29]:
######## for clean data ##########

from deeprobust.graph.defense import RGCN

model = RGCN(nnodes=adj.shape[0], nfeat=features.shape[1], nclass=labels.max()+1, nhid=32, device='cpu')

model.fit(features, adj, labels, idx_train, idx_val, train_iters=200, verbose=True)

model.test(idx_test)

=== training rgcn model ===
Epoch 0, training loss: 8.522789001464844
Epoch 10, training loss: 2.359901189804077
Epoch 20, training loss: 2.037491798400879
Epoch 30, training loss: 1.9117820262908936
Epoch 40, training loss: 1.6561987400054932
Epoch 50, training loss: 1.595266580581665
Epoch 60, training loss: 1.4696218967437744
Epoch 70, training loss: 1.4231951236724854
Epoch 80, training loss: 1.4062306880950928
Epoch 90, training loss: 1.3625150918960571
Epoch 100, training loss: 1.324859857559204
Epoch 110, training loss: 1.313098669052124
Epoch 120, training loss: 1.2610596418380737
Epoch 130, training loss: 1.2950108051300049
Epoch 140, training loss: 1.3298710584640503
Epoch 150, training loss: 1.2866679430007935
Epoch 160, training loss: 1.2568919658660889
Epoch 170, training loss: 1.2359753847122192
Epoch 180, training loss: 1.231086254119873
Epoch 190, training loss: 1.2314555644989014
=== picking the best model according to the performance on validation ===
Test set results

0.8390342052313883

In [30]:
######## for perturbed_adj data ##########

from deeprobust.graph.defense import RGCN

model = RGCN(nnodes=perturbed_adj.shape[0], nfeat=features.shape[1], nclass=labels.max()+1, nhid=32, device='cpu')

model.fit(features, perturbed_adj, labels, idx_train, idx_val, train_iters=200, verbose=True)

model.test(idx_test)

=== training rgcn model ===
Epoch 0, training loss: 8.680205345153809
Epoch 10, training loss: 2.492426633834839
Epoch 20, training loss: 2.1194214820861816
Epoch 30, training loss: 1.9442079067230225
Epoch 40, training loss: 1.7106220722198486
Epoch 50, training loss: 1.5851284265518188
Epoch 60, training loss: 1.5247790813446045
Epoch 70, training loss: 1.4218220710754395
Epoch 80, training loss: 1.4154812097549438
Epoch 90, training loss: 1.3573063611984253
Epoch 100, training loss: 1.3644789457321167
Epoch 110, training loss: 1.3453865051269531
Epoch 120, training loss: 1.3252500295639038
Epoch 130, training loss: 1.2749245166778564
Epoch 140, training loss: 1.2743579149246216
Epoch 150, training loss: 1.265657663345337
Epoch 160, training loss: 1.2316855192184448
Epoch 170, training loss: 1.2856961488723755
Epoch 180, training loss: 1.2764875888824463
Epoch 190, training loss: 1.2590816020965576
=== picking the best model according to the performance on validation ===
Test set res

0.8259557344064387

In [31]:
######################## GLet Attack without modified feature ####################

from deeprobust.graph.defense import RGCN

model = RGCN(nnodes=mod_adj.shape[0], nfeat=features.shape[1], nclass=labels.max()+1, nhid=32, device='cpu')

model.fit(features, mod_adj, labels, idx_train, idx_val, train_iters=200, verbose=True)

model.test(idx_test)

=== training rgcn model ===
Epoch 0, training loss: 8.542876243591309
Epoch 10, training loss: 2.4007084369659424
Epoch 20, training loss: 2.002580404281616
Epoch 30, training loss: 1.9172433614730835
Epoch 40, training loss: 1.6416516304016113
Epoch 50, training loss: 1.5304112434387207
Epoch 60, training loss: 1.4558850526809692
Epoch 70, training loss: 1.3763169050216675
Epoch 80, training loss: 1.4914183616638184
Epoch 90, training loss: 1.3599382638931274
Epoch 100, training loss: 1.3390189409255981
Epoch 110, training loss: 1.2999459505081177
Epoch 120, training loss: 1.2953813076019287
Epoch 130, training loss: 1.2818228006362915
Epoch 140, training loss: 1.2846062183380127
Epoch 150, training loss: 1.243011236190796
Epoch 160, training loss: 1.2191160917282104
Epoch 170, training loss: 1.2832551002502441
Epoch 180, training loss: 1.2242143154144287
Epoch 190, training loss: 1.2374404668807983
=== picking the best model according to the performance on validation ===
Test set res

0.8329979879275654

In [ ]:
############ ############ GLet Attack with modified feature  ########## ##########

from deeprobust.graph.defense import RGCN

model = RGCN(nnodes=mod_adj.shape[0], nfeat=mod_fet.shape[1], nclass=labels.max()+1, nhid=32, device='cpu')

#model.fit(mod_fet, mod_adj, labels, idx_train, idx_val, train_iters=200, verbose=True)
#model.test(idx_test)

=== training rgcn model ===
Epoch 0, training loss: 8.377226829528809
Epoch 10, training loss: 2.558807849884033
Epoch 20, training loss: 2.1916861534118652
Epoch 30, training loss: 2.063342571258545
Epoch 40, training loss: 1.9379050731658936
Epoch 50, training loss: 1.7629766464233398
Epoch 60, training loss: 1.7432069778442383
Epoch 70, training loss: 1.6723780632019043
Epoch 80, training loss: 1.6380817890167236
Epoch 90, training loss: 1.651607632637024
Epoch 100, training loss: 1.618840217590332
Epoch 110, training loss: 1.5954811573028564
Epoch 120, training loss: 1.6019055843353271
Epoch 130, training loss: 1.5915484428405762
Epoch 140, training loss: 1.5970122814178467
Epoch 150, training loss: 1.5887224674224854
Epoch 160, training loss: 1.5663660764694214
Epoch 170, training loss: 1.626084804534912
Epoch 180, training loss: 1.6050693988800049
Epoch 190, training loss: 1.5854681730270386
=== picking the best model according to the performance on validation ===
Test set result

0.8199195171026157

**Comparison of GNN and Random Forest**

In [32]:
import numpy as np

# create adjacency matrix
#adj_matrix = np.array(mod_adj)

adj_matrix = mod_adj.toarray()

# get the indices of the non-zero elements
indices = np.where(adj_matrix != 0)

# create an edge list from the indices
edge_list = list(zip(indices[0], indices[1]))

print(edge_list) 

[(0, 1084), (0, 1104), (0, 1288), (0, 1496), (0, 2218), (1, 1147), (1, 1504), (1, 1705), (1, 2205), (2, 162), (3, 2470), (4, 1566), (4, 1981), (5, 6), (5, 1971), (6, 5), (6, 2381), (6, 2396), (7, 8), (8, 7), (9, 165), (9, 2218), (10, 14), (10, 165), (10, 1271), (11, 13), (11, 164), (11, 423), (11, 691), (11, 1005), (11, 1019), (11, 1080), (11, 1201), (11, 1212), (11, 1319), (11, 1505), (11, 1621), (11, 1624), (11, 1759), (11, 1830), (11, 2044), (11, 2050), (11, 2051), (11, 2057), (11, 2242), (11, 2369), (12, 16), (13, 11), (13, 247), (13, 841), (13, 842), (13, 844), (13, 862), (13, 1005), (13, 1034), (13, 1301), (13, 1372), (13, 1746), (13, 2171), (13, 2195), (13, 2198), (13, 2445), (14, 10), (14, 738), (14, 852), (14, 995), (14, 1271), (14, 2218), (15, 16), (15, 37), (15, 57), (15, 188), (15, 228), (15, 279), (15, 384), (15, 1322), (15, 1742), (15, 1826), (15, 2060), (16, 12), (16, 15), (16, 943), (16, 1265), (16, 2281), (16, 2376), (17, 1785), (18, 1524), (19, 1338), (19, 1550), (19,

In [33]:
my_tensor = torch.tensor(edge_list)

In [34]:
len(my_tensor)

9630

In [35]:
import numpy
np_graph_ed = my_tensor.numpy()

In [36]:
x=len(pyg_data[0].y)
y=len(np_graph_ed)
xy=[x,y]

/usr/local/lib/python3.9/dist-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [37]:
xy

[2485, 9630]

In [38]:
with open("graph.in","w") as f:
    f.write("\n".join(" ".join(map(str, x)) for x in (np_graph_ed)))

In [39]:
with open('graph.in', 'r') as file:
    contents = file.read()
    file.close()

with open('graph.in', 'w') as file:
    file.write(' '.join(map(str, xy)))
    file.write("\n")
    file.write(contents)
    file.close()

In [40]:
!python /content/drive/MyDrive/orca-py2/orca-py/test.py

graph: Graph with 2485 nodes and 4815 edges
Number of nodes: 2485
Number of graphlets: 73
Counts:  [[5, 47, 8, 2, 148, 143, 425, 5, 1, 14, 41, 4, 1, 1, 0, 863, 411, 487, 656, 1990, 1273, 137, 3416, 1, 55, 104, 81, 30, 42, 152, 15, 264, 389, 2, 6, 17, 1, 8, 2, 4, 30, 3, 2, 13, 0, 9, 3, 2, 34, 0, 0, 0, 4, 1, 0, 0, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5, 14, 8, 2, 122, 49, 15, 5, 1, 7, 5, 4, 0, 1, 0, 419, 402, 63, 915, 213, 54, 47, 5, 1, 102, 0, 9, 36, 24, 61, 17, 9, 5, 2, 1, 8, 10, 1, 2, 3, 0, 0, 2, 1, 0, 7, 0, 0, 5, 0, 0, 3, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [7, 80, 16, 5, 258, 388, 921, 14, 4, 16, 84, 17, 0, 4, 0, 1509, 1261, 1805, 872, 5494, 4533, 555, 8380, 4, 88, 179, 260, 92, 68, 237, 230, 376, 773, 15, 10, 27, 12, 208, 12, 3, 0, 37, 10, 24, 3, 13, 0, 0, 78, 6, 0, 4, 1, 8, 0, 0, 2, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [31, 72, 453, 12, 444, 2054, 73, 4154, 19, 17, 46, 336, 11, 3, 2, 1641, 12021, 2218, 3359, 895, 2039, 27607,

In [41]:
with open('graph.out', 'r') as file:
    gl = file.readlines()

In [42]:
int_list = [[int(num) for num in s.strip('[]\n').split(',')] for s in gl]
graphlet_features = np.array(int_list)
print(graphlet_features)

[[ 5 47  8 ...  0  0  0]
 [ 5 14  8 ...  0  0  0]
 [ 7 80 16 ...  0  0  0]
 ...
 [ 1  3  0 ...  0  0  0]
 [ 1  3  0 ...  0  0  0]
 [ 1  2  0 ...  0  0  0]]


In [43]:
import pandas as pd
# create a list of column names
column_names = ['X'+ str(i) for i in range(len(graphlet_features[0]))]
df = pd.DataFrame(graphlet_features, columns=column_names)
# add a new column to the DataFrame

In [44]:
df['Class'] = np.array(pyg_data[0].y)

/usr/local/lib/python3.9/dist-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [45]:
df

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X64,X65,X66,X67,X68,X69,X70,X71,X72,Class
0,5,47,8,2,148,143,425,5,1,14,...,0,0,0,0,0,0,0,0,0,5
1,5,14,8,2,122,49,15,5,1,7,...,0,0,0,0,0,0,0,0,0,2
2,7,80,16,5,258,388,921,14,4,16,...,0,0,0,0,0,0,0,0,0,0
3,31,72,453,12,444,2054,73,4154,19,17,...,0,3,1,1,0,0,2,0,0,1
4,5,20,10,0,192,78,31,10,1,2,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480,4,4,3,3,91,6,0,0,0,0,...,0,0,0,0,0,0,1,0,0,6
2481,1,41,0,0,89,0,807,0,0,13,...,0,0,0,0,0,0,0,0,0,0
2482,1,3,0,0,23,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2483,1,3,0,0,23,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [46]:
# split data into input and taget variable(s)
X = df.drop("Class", axis=1)
y = df["Class"]

In [47]:
# import important packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas_profiling

from matplotlib import rcParams
import warnings

warnings.filterwarnings("ignore")

# figure size in inches
rcParams["figure.figsize"] = 10, 6
np.random.seed(42)

In [48]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.20, random_state=42)

In [49]:
# Building the Random Forest Classifier
# Now is time to create our random forest classifier and then train it on the train set. 
# We will also pass the number of trees (300) in the forest we want to use through the parameter
# called n_estimators.

# create the classifier
classifier = RandomForestClassifier(n_estimators=300)

# Train the model using the training sets
classifier.fit(X_train, y_train)

# predictin on the test set
y_pred = classifier.predict(X_test)

# Calculate Model Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.21327967806841047


**taking only node feature**

In [50]:
# add a new column to the DataFrame
feature = np.array(pyg_data[0].x)

In [51]:
len(feature)

2485

****Merge Node Feature and Graphlet Orbit Feature****

In [52]:
# create a list of column names
column_names = ['X'+ str(i) for i in range(len(graphlet_features[0]))]
df1 = pd.DataFrame(graphlet_features, columns=column_names)
# add an index column
df1 = df1.reset_index()

In [53]:
# create a list of column names
column_names = ['Xf'+ str(i) for i in range(len(feature[0]))]

In [54]:
df2 = pd.DataFrame(feature, columns=column_names)
# add an index column
df2 = df2.reset_index()

In [55]:
# merge the data frames based on the 'key' column
merged_df = pd.merge(df1, df2, on='index')
# split data into input and taget variable(s)
merged_df = merged_df.drop("index", axis=1)
# add a new column to the DataFrame
merged_df['Class'] = np.array(pyg_data[0].y)

In [56]:
# split data into input and taget variable(s)
X = merged_df.drop("Class", axis=1)
y = merged_df["Class"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split( X_scaled, y, stratify=y, test_size=0.20, random_state=42)

In [57]:
# Building the Random Forest Classifier
# Now is time to create our random forest classifier and then train it on the train set. 
# We will also pass the number of trees (300) in the forest we want to use through the parameter
# called n_estimators.

# create the classifier
classifier = RandomForestClassifier(n_estimators=300)

# Train the model using the training sets
classifier.fit(X_train, y_train)

# predictin on the test set
y_pred = classifier.predict(X_test)

# Calculate Model Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.6901408450704225


**Central Node**

In [58]:
target= np.array(pyg_data[0].y)

In [59]:
dfcn = pd.DataFrame(target, columns=['Class'])
######## Taking only central orbit features #########

# add a new column to the DataFrame
dfcn['X23'] = np.array(graphlet_features[:, 23])
# add a new column to the DataFrame
dfcn['X33'] = np.array(graphlet_features[:, 33])
# add a new column to the DataFrame
dfcn['X44'] = np.array(graphlet_features[:, 44])
# add a new column to the DataFrame
dfcn['X69'] = np.array(graphlet_features[:, 69])

In [60]:
# split data into input and taget variable(s)
X = dfcn.drop("Class", axis=1)
y = dfcn["Class"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split( X_scaled, y, stratify=y, test_size=0.20, random_state=42)

# create the classifier
classifier = RandomForestClassifier(n_estimators=300)
# Train the model using the training sets
classifier.fit(X_train, y_train)

# predictin on the test set
y_pred = classifier.predict(X_test)
# Calculate Model Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.2414486921529175


**central graphlet orbit plus x node feature**

In [61]:
# add an index column
dfcn = dfcn.reset_index()

In [63]:
# merge the data frames based on the 'key' column
merged_df = pd.merge(dfcn, df2, on='index')

In [64]:
# split data into input and taget variable(s)
merged_df = merged_df.drop("index", axis=1)

# split data into input and taget variable(s)
X = merged_df.drop("Class", axis=1)
y = merged_df["Class"]

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split( X_scaled, y, stratify=y, test_size=0.20, random_state=42)

# create the classifier
classifier = RandomForestClassifier(n_estimators=300)
# Train the model using the training sets
classifier.fit(X_train, y_train)

# predictin on the test set
y_pred = classifier.predict(X_test)
# Calculate Model Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.738430583501006
